# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링 (목표 user 1000명)

- user_profile_df : User의 아이디, 리뷰수, 팔로워, url을 저장함
- user_df : '아이디', '리뷰', '팔로워', '매장명', '카테고리', '주소', '리뷰 내용', '세부정보', '태그', '방문일자', '재방문횟수' 를 저장함


### 문제

1. 인기 리뷰어라면 핫플레이스만 다녀서 평범한 음식점에 대한 정보는 못얻지 않을까?
2. 리뷰어는 어디서 가져오지?
3. 연도를 어떻게 하지 ? 연도가 없는 경우도 있고, 연도와 요일이 없는 경우도 있음 

In [18]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  

from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import os
from datetime import datetime

import pandas as pd
from selenium.webdriver.common.by import By

### 1. User 정보를 가져오기 위해 (현재 기준 없음) user의 myplace url을 복사한다. 

In [19]:
url = 'https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2'


User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 

In [20]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()

# User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 
def user_profile(url) :
    # 주소 이동
    driver.get(url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    user_element = {}

    # User ID 찾기
    user_id_element = soup.find('button', class_='wTaI4v _2kK3N- _2we3hB')
    user_element['아이디'] = user_id_element.text if user_id_element else None

    # User의 인기도 : 리뷰수, 팔로워 찾기 
    user_popularity = soup.find_all('button', class_='wTaI4v _15qVKh')
    
    for element in user_popularity:
        em_tag = element.find('em')
        if em_tag:
            key = element.text.replace(em_tag.text, '').strip()
            value = int(re.sub('[^0-9]', '', em_tag.text))  # 쉼표 제거 후 변환
            user_element[key] = value
    
    # User URL
    user_element['주소'] = url

    print(user_element)

    return user_element


일자, 요일, 방문일수 분류 함수 

In [21]:
def change_date_format(날짜) :
    #  부분 추출 (년월일까지)
    date_part = ""
    for i in 날짜 :
        date_part += i
        date_part += ','
    date_part = date_part.split('\n')[1]

    # 일자, 요일, 방문일수 분류하기 
    date_part = date_part.split(',')
    date = date_part[0]
    weekday = date[-3:]
    revisit = int(date_part[1].replace('번째 방문', ''))

    # 일자 타입 변경하기 
    date = '24' + date if date.startswith('년') else date # 만약 연도가 없는 경우 24를 붙이기
    match = re.findall(r'(\d+)년 (\d+)월 (\d+)일', date) # 타입 바꾸기 
    year, month, day = map(int, match[0])

    # 날짜 객체로 변환
    date_object = datetime(2000+year, month, day) 
    formatted_date = date_object.strftime('%Y-%m-%d')

    return formatted_date, weekday, revisit



### 유저 정보 찾기

In [22]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()

# User의 아이디, 리뷰수, 팔로워를 가져오는 함수 
user_data = user_profile(url)

# User 정보 저장 
user_profile_df = pd.DataFrame([user_data], index=[0])
user_profile_df

{'아이디': 'xll****', '리뷰': 983, '팔로잉': 0, '팔로워': 75, '주소': 'https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2'}


,아이디,리뷰,팔로잉,팔로워,주소
0,xll****,983,0,75,https://m.place.naver.com/my/5c36b9f1e511a8856...


### 매장 정보 찾기

In [23]:
# 맨 처음 게시물 클릭 
button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
button.click()
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [24]:
# data를 담을 빈 리스트 선언
data_list = [] 

# 페이지 아래로 스크롤 몇번 
page_down = 5

# 페이지 스크롤 횟수만큼 반복
for _ in range(page_down):

    # 대기시간 5초 
    wait = WebDriverWait(driver, 5)

    # 요소를 찾을 때 대기 조건 추가
    user_review_elements = wait.until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92'))
    )

    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 

    # 리뷰에서 정보 가져오기
    for reviews_elements in user_review_elements:
        i = 0 # 카테고리와 주소 구분할 때 사용
        restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')
        
        if bool(restaurant_elements): # 정보가 없는 경우가 있음

            # 매장명 찾기
            restaurant_name = restaurant_elements[0].text  

    #<--------------------------------------------------------------------------------------------------->
            # 카테고리와 주소 찾기 
            category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
            category_location_soup = BeautifulSoup(category_location_elements[i].get_attribute('outerHTML'), 'html.parser')
            
            # span 태그 안에 있는 텍스트 가져오기
            span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
            category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
            location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
            i += 1
    #<--------------------------------------------------------------------------------------------------->
            # 리뷰 찾기 
            review_elements = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')
            try : 
                review_text = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')[0].text # 리뷰 본문          
                sub_info=reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') #세부정보
                if bool(sub_info):
                    sub_info=sub_info[0].text
                else:
                    sub_info=None
            except :
                review_text = '리뷰 없음'
                sub_info = ''

    #<--------------------------------------------------------------------------------------------------->
            # 태그 찾기 
            # 일정 개수가 넘어가면 리뷰가 숨겨져 표시되므로 
            reactions_elements=reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트
            command=False
            for x in reactions_elements:
                if x.get_attribute("role"): # 리액션 숨김 없애기
                    x.click()
                    command=True
                else:
                    continue
            if command:
                reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트 재탐색
            reactions=list(map(lambda x:x.text,reactions_elements))
            # 리스트를 벗김 
            reactions = str(reactions).replace('[', '').replace(']', '')

    #<--------------------------------------------------------------------------------------------------->
            # 방문 일자, 재방문 이력 찾기
            date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
            date=[x.text for x in date_info]
            day,weekday,revisit = change_date_format(date) # 함수 사용
        
            # 정보 추가
            data_dict = {
                    '아이디' : user_data['아이디'],
                    '리뷰' : user_data['리뷰'],
                    '팔로워' : user_data['팔로워'],
                    '매장명': restaurant_name,
                    '카테고리': category,
                    '주소' : location,
                    '리뷰 내용': review_text,
                    "세부정보" : sub_info,
                    '태그': reactions,
                    '방문일자': day,
                    '요일' : weekday,
                    '재방문횟수':revisit}
            data_list.append(data_dict)
    user_df = pd.DataFrame(data_list)
    user_df = user_df.drop_duplicates()
  
user_df.head(5)

,아이디,리뷰,팔로워,매장명,카테고리,주소,리뷰 내용,세부정보,태그,방문일자,요일,재방문횟수
0,xll****,983,75,매우매오 강남역별관,"육류,고기요리",서울특별시 강남구 역삼동,평이 계속 왔다갔다했는데 일단 세트 구성과 끓이면서 따끈하게 먹을 수 있는게 좋아요...,이용 방법예약 없이대기 시간바로 입장동행친구,'메뉴 구성이 알차요',2024-02-13,화요일,1
1,xll****,983,75,다람제과,"카페,디저트",서울특별시 마포구 신공덕동,맘모스치즈케이크 포장하면 소보로?올려주십니다! 눅눅해지지않게ㅎㅎ 요새 빵가격이 올랐...,이용 방법예약 없이대기 시간바로 입장,"'디저트가 맛있어요', '특별한 메뉴가 있어요'",2024-01-26,금요일,1
2,xll****,983,75,플러피 베이크샵,"카페,디저트",서울특별시 마포구 공덕동,독특한 메뉴가 많아서 좋아요 친절하세요:) 옆집보다 감태향이 강하고 옛날김과자?느낌...,이용 방법예약 없이대기 시간바로 입장목적일상동행혼자,"'특별한 메뉴가 있어요', '친절해요'",2024-01-26,금요일,1
3,xll****,983,75,뺑스톡 공덕점,베이커리,서울특별시 마포구 공덕동,좌석도 있네요. 감태휘낭시에 처음봐서 먹어봤는데 옆집보다 감태향은 낮지만 대신 짭짤...,이용 방법예약 없이대기 시간바로 입장,"'빵이 맛있어요', '특별한 메뉴가 있어요'",2024-01-26,금요일,1
4,xll****,983,75,맘스터치 굽은다리역점,햄버거,서울특별시 강동구 천호동,떡강정먹기 좋아요! 점심시간대여서인지 배달도 많고 포장인데 15-20분걸렸으니 참고...,이용 방법포장·배달대기 시간30분 이내,'음식이 맛있어요',2024-01-25,목요일,1


유저 리뷰 가져오는 함수

In [25]:
def find_user_data(page_down,driver) : 
    # 맨 처음 게시물 클릭 
    button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
    button.click()
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # data를 담을 빈 리스트 선언
    data_list = [] 


    # 페이지 스크롤 횟수만큼 반복
    for _ in range(page_down):

        # 대기시간 5초 
        wait = WebDriverWait(driver, 5)

        # 요소를 찾을 때 대기 조건 추가
        user_review_elements = wait.until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92'))
        )

        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 

        # 리뷰에서 정보 가져오기
        for reviews_elements in user_review_elements:
            i = 0 # 카테고리와 주소 구분할 때 사용
            restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')
            
            if bool(restaurant_elements): # 정보가 없는 경우가 있음

                # 매장명 찾기
                restaurant_name = restaurant_elements[0].text  

        #<--------------------------------------------------------------------------------------------------->
                # 카테고리와 주소 찾기 
                category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
                category_location_soup = BeautifulSoup(category_location_elements[i].get_attribute('outerHTML'), 'html.parser')
                
                # span 태그 안에 있는 텍스트 가져오기
                span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
                category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
                location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
                i += 1
        #<--------------------------------------------------------------------------------------------------->
                # 리뷰 찾기 
                review_elements = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')
                try : 
                    review_text = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')[0].text # 리뷰 본문          
                    sub_info=reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') #세부정보
                    if bool(sub_info):
                        sub_info=sub_info[0].text
                    else:
                        sub_info=None
                except :
                    review_text = '리뷰 없음'
                    sub_info = ''

        #<--------------------------------------------------------------------------------------------------->
                # 태그 찾기 
                # 일정 개수가 넘어가면 리뷰가 숨겨져 표시되므로 
                reactions_elements=reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트
                command=False
                for x in reactions_elements:
                    if x.get_attribute("role"): # 리액션 숨김 없애기
                        x.click()
                        command=True
                    else:
                        continue
                if command:
                    reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트 재탐색
                reactions=list(map(lambda x:x.text,reactions_elements))
                # 리스트를 벗김 
                reactions = str(reactions).replace('[', '').replace(']', '')

        #<--------------------------------------------------------------------------------------------------->
                # 방문 일자, 재방문 이력 찾기
                date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
                date=[x.text for x in date_info]
                day,weekday,revisit = change_date_format(date) # 함수 사용
            
                # 정보 추가
                data_dict = {
                        '아이디' : user_data['아이디'],
                        '리뷰' : user_data['리뷰'],
                        '팔로워' : user_data['팔로워'],
                        '매장명': restaurant_name,
                        '카테고리': category,
                        '주소' : location,
                        '리뷰 내용': review_text,
                        "세부정보" : sub_info,
                        '태그': reactions,
                        '방문일자': day,
                        '요일' : weekday,
                        '재방문횟수':revisit}
                data_list.append(data_dict)
        user_df = pd.DataFrame(data_list)
        user_df = user_df.drop_duplicates()
  
    return user_df



# 위 내용을 반복문으로 한번에 하기 

In [26]:
# 네이버에서 찾아온 리뷰어들의 주소 
# myplace url
user_list = ['https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2',
'https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2',
# 'https://m.place.naver.com/my/5f1dd9049ec8258e4a657f78/review?v=2',
# 'https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2',
# 'https://m.place.naver.com/my/6010e880e71246c530be8c27/review?v=2',
# 'https://m.place.naver.com/my/5e27e4ec8f87a842bcb3505c/review?v=2',
'https://m.place.naver.com/my/5cf4c309c1dd7fdcdfcd76ac/review?v=2']


# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()
# 스크롤 횟수 
page_down = 3

# 데이터프레임 선언
profile_df = pd.DataFrame()
total_user_df =  pd.DataFrame()

# 반복해서 url 정보를 가져오자~ 
for url in user_list :
    try :     
        # User의 아이디, 리뷰수, 팔로워를 가져오는 함수 
        user_data = user_profile(url)
        # user_profile_df = pd.DataFrame([user_data], index=[0])
        if 'user_profile_df' in locals() and not user_profile_df.empty:
            user_profile_df = user_profile_df.append(user_data, ignore_index=True)
        else:
            user_profile_df = pd.DataFrame([user_data], index=[0])
            
        # User review 정보를 가져오는 함수
        user_df = find_user_data(page_down,driver)

        # User review 정보 저장 
        total_user_df = pd.concat([total_user_df, user_df], ignore_index=True)

    except Exception as e:
        print(f"에러 메시지: {str(e)}")
        continue

{'아이디': 'xll****', '리뷰': 983, '팔로잉': 0, '팔로워': 75, '주소': 'https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2'}
{'아이디': '맛있는거 먹으려고 운동함', '리뷰': 534, '팔로잉': 2, '팔로워': 42, '주소': 'https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2'}
{'아이디': '포도267', '리뷰': 665, '팔로잉': 0, '팔로워': 13, '주소': 'https://m.place.naver.com/my/5cf4c309c1dd7fdcdfcd76ac/review?v=2'}


KeyboardInterrupt: 

In [ ]:
user_profile_df

,아이디,리뷰,팔로잉,팔로워,주소
0,포도267,665,0,13,https://m.place.naver.com/my/5cf4c309c1dd7fdcd...
1,xll****,983,0,75,https://m.place.naver.com/my/5c36b9f1e511a8856...
2,맛있는거 먹으려고 운동함,534,2,42,https://m.place.naver.com/my/5e1370ce8f87a842b...
3,포도267,665,0,13,https://m.place.naver.com/my/5cf4c309c1dd7fdcd...


In [ ]:
total_user_df

,아이디,리뷰,팔로워,매장명,카테고리,주소,리뷰 내용,세부정보,태그,방문일자,요일,재방문횟수
0,xll****,983,75,매우매오 강남역별관,"육류,고기요리",서울특별시 강남구 역삼동,평이 계속 왔다갔다했는데 일단 세트 구성과 끓이면서 따끈하게 먹을 수 있는게 좋아요...,이용 방법예약 없이대기 시간바로 입장동행친구,'메뉴 구성이 알차요',2024-02-13,화요일,1
1,xll****,983,75,다람제과,"카페,디저트",서울특별시 마포구 신공덕동,맘모스치즈케이크 포장하면 소보로?올려주십니다! 눅눅해지지않게ㅎㅎ 요새 빵가격이 올랐...,이용 방법예약 없이대기 시간바로 입장,"'디저트가 맛있어요', '특별한 메뉴가 있어요'",2024-01-26,금요일,1
2,xll****,983,75,플러피 베이크샵,"카페,디저트",서울특별시 마포구 공덕동,독특한 메뉴가 많아서 좋아요 친절하세요:) 옆집보다 감태향이 강하고 옛날김과자?느낌...,이용 방법예약 없이대기 시간바로 입장목적일상동행혼자,"'특별한 메뉴가 있어요', '친절해요'",2024-01-26,금요일,1
3,xll****,983,75,뺑스톡 공덕점,베이커리,서울특별시 마포구 공덕동,좌석도 있네요. 감태휘낭시에 처음봐서 먹어봤는데 옆집보다 감태향은 낮지만 대신 짭짤...,이용 방법예약 없이대기 시간바로 입장,"'빵이 맛있어요', '특별한 메뉴가 있어요'",2024-01-26,금요일,1
4,xll****,983,75,맘스터치 굽은다리역점,햄버거,서울특별시 강동구 천호동,떡강정먹기 좋아요! 점심시간대여서인지 배달도 많고 포장인데 15-20분걸렸으니 참고...,이용 방법포장·배달대기 시간30분 이내,'음식이 맛있어요',2024-01-25,목요일,1
5,xll****,983,75,동백식탁,이탈리아음식,경기도 용인시 기흥구 중동,친구가 맛있다고 데려왔는데 맛있어요! 셀프바가 있고 남은거 포장도 됩니다~,,"'음식이 맛있어요', '친절해요', '매장이 청결해요'",2024-01-22,월요일,1
6,xll****,983,75,판다월드샵,"판촉,기념품",경기도 용인시 처인구 포곡읍,푸바오 러바오 만나고 기념품샀어요ㅎㅎ 귀엽네요\n예상보다 가격대 괜찮았어요,None,"'종류가 다양해요', '특색 있는 제품이 많아요', '아기자기해요', '친절해요'",2024-01-22,월요일,1
7,xll****,983,75,함지박,중식당,경기도 용인시 수지구 풍덕천동,바가지탕수육 고기두툼하고 맛도 좋아요! 바가지빵도 부숴먹었는데 얇은 페스츄리느낌이예...,이용 방법예약 없이대기 시간바로 입장목적친목,"'음식이 맛있어요', '특별한 메뉴가 있어요', '친절해요'",2024-01-22,월요일,1
8,xll****,983,75,디어필립,베이커리,경기도 용인시 수지구 풍덕천동,소세지맛이 강했고 에그타르트 촉촉하게 잘 먹었어요! 빵얘기하면 담아주십니다~,이용 방법예약 없이대기 시간바로 입장,"'빵이 맛있어요', '특별한 메뉴가 있어요'",2024-01-22,월요일,1
9,xll****,983,75,하구영베이커리,베이커리,경기도 용인시 수지구 풍덕천동,밤많이파이 잘 안보이던 타입이라 사봤는데 너무 맘에 들어요! 바밤바같은 밤맛보다 담...,이용 방법예약 없이대기 시간바로 입장,"'빵이 맛있어요', '특별한 메뉴가 있어요'",2024-01-22,월요일,1


In [ ]:
# 유저의 리뷰
user_df.to_excel('user_df.xlsx', index=False)

# 유저의 정보 
user_profile_df.to_excel('user_profile_df.xlsx', index=False)

In [ ]:
# 리셋
data_list = []
del user_profile_df
del total_user_df